In [2]:
#Install the neo4j driver:
!pip install neo4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.3/176.3 kB 7.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.7.0-py3-none-any.whl size=243954 sha256=3f6d7594de7ffaffd649bd933172b6a7cc0b46e49db43990362603769b750a9f
  Stored in directory: /root/.cache/pip/wheels/6d/be/1e/749b84bba4b7429145ea09ccb46114fab4748625eb319f286a
Successfully built neo4j


In [3]:
#Import the GraphDatabase.driver module:
from neo4j import GraphDatabase

In [4]:
# Import required libraries
from neo4j import GraphDatabase,basic_auth
import pandas as pd
import psycopg2

import logging
#Setup logger
logging.basicConfig(filename='pipeline.log', level=logging.DEBUG)

In [6]:
# Define Neo4j connection details
neo4j_uri = "neo4j+s://968e72f6.databases.neo4j.io"
neo4j_user = "neo4j"
neo4j_password = "VdUIagcWA11O3v7PxrmpYQEG67ayEjUyqwRQN4m_dmg"

In [7]:
# Postgres Database Information [running on my PC]
pg_host = 'localhost'
pg_database = 'postgres'
pg_user = 'postgres'
pg_password = 'pass12345'

In [8]:
# Define Neo4j query to extract data
neo4j_query = "MATCH (c:customer_data) RETURN c"

In [9]:
# Define function to extract data from Neo4j and return a Pandas DataFrame
def extract_data(uri, user, pwd, q):
    
    """
    Extract function that connects to Neo4j database and returns records from the customer_data node in pandas dataframe
    """
    # Connect to Neo4j
    try:
        driver = GraphDatabase.driver(uri, auth=(user, pwd))

        with driver.session() as session:
            query = q
            results = session.run(query)
            df = pd.DataFrame([dict(record['c']) for record in results])

        driver.close()
    except Exception as e:
        err = "Extract() error - "+str(e)
        logging.debug(err)
        
        
    return df

In [10]:
# Define function to transform data
def transform_data(df):
    
    """
    Transform function that converts date columns from string to datetime format, drops unneeded column and null values 
    Function returns cleaned df
    """
    # Convert date fields to datetime objects
    try:
        
        df["start_date"] = pd.to_datetime(df["start_date"],format='%d-%m-%Y')
        df["end_date"] = pd.to_datetime(df["end_date"],format='%d-%m-%Y')

        #drop date of birth column
        df = df.drop('date_of_birth', axis=1)

        # Remove null values
        df = df.dropna()
    
    except Exception as e:
        err = "Transform() error - "+str(e)
        logging.debug(err)
     
    return df


In [11]:
# Define function to load data into Postgres
def load_data(transformed_df):
    
    """
    Function connects to local postgres db instance running on docker desktop
    It creates table and uploads contents of transformed df into the table
    """
    # Connect to Postgres
    try:
        
        conn = psycopg2.connect(host=pg_host, database=pg_database, user=pg_user, password=pg_password)
        # Create table if it doesn't exist
        with conn.cursor() as cursor:
            cursor.execute("""
            CREATE TABLE IF NOT EXISTS telecom_data (
                customer_id INTEGER,
                subscription_id INTEGER,
                service_id VARCHAR,
                start_date DATE,
                end_date DATE,
                price FLOAT
            )
            """)
      
            for _, row in transformed_df.iterrows():
                cursor.execute("INSERT INTO telecom_data (customer_id, subscription_id, service_id, start_date, end_date, price) VALUES (%s, %s, %s, %s, %s, %s)",
                       (row['customer_id'], row['subscription_id'], row['service_id'], row['start_date'], row['end_date'], row['subscription_price']))


        conn.commit()


        # Close the cursor and connection
        cursor.close()
        conn.close()
    
    except Exception as e:
        err = "Load() error - "+str(e)
        logging.debug(err)
        

In [12]:
# Define main function
def main():
    # Extract data from Neo4j
    df = extract_data(neo4j_uri, neo4j_user, neo4j_password, neo4j_query)
    
    # Transform data using Pandas
    df = transform_data(df)
    
    # Load data into Postgres
    load_data(df)

In [13]:
# Call main function
if __name__ == "__main__":
    main()